<a href="https://colab.research.google.com/github/SepKfr/Corruption-resilient-Forecasting-Models/blob/master/example_run.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/SepKfr/Corruption-resilient-Forecasting-Models.git

In [ ]:
%cd Corruption-resilient-Forecasting-Models/

In [73]:
!bash run.sh

Using root folder /content/Corruption-resilient-Forecasting-Models/data/outputs
Formatting data.
Setting scalers with data_set...
[I 2023-01-30 22:20:09,061] A new study created in memory with name: ATA_gp
Train epoch: 0, loss: 124.6345
val loss: 14.5438
Train epoch: 5, loss: 102.1675
val loss: 11.9817
Train epoch: 10, loss: 79.0824
val loss: 9.0511
Train epoch: 15, loss: 71.3089
val loss: 8.3995
Train epoch: 20, loss: 65.9213
val loss: 7.9061
/usr/local/lib/python3.8/dist-packages/gpytorch/distributions/multivariate_normal.py:319: NumericalWarning: Negative variance values detected. This is likely due to numerical instabilities. Rounding negative variances up to 1e-06.
  warnings.warn(
Train epoch: 25, loss: 61.2346
val loss: 7.5152
Train epoch: 30, loss: 57.6559
val loss: 7.2874
Train epoch: 35, loss: 54.7390
val loss: 7.1164
Train epoch: 40, loss: 52.9050
val loss: 7.0363
Train epoch: 45, loss: 51.0077
val loss: 6.6284
[I 2023-01-30 22:26:13,714] Trial 0 finished with value: 6.52326

In [ ]:
%tb

import argparse
import json
import random
from collections import OrderedDict

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import os

from Utils.base_train import batching, ModelData, batch_sampled_data
from data_loader import ExperimentConfig
from forecast_denoising import Forecast_denoising

exp_name = "solar"

def evaluate(model_name, denoising, gp):


    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    pred_len = 24
    attn_type = "ATA"
    n_heads = 8
    d_model = [16, 32]
    batch_size = 256

    config = ExperimentConfig(pred_len, exp_name)

    formatter = config.make_data_formatter()
    params = formatter.get_experiment_params()
    total_time_steps = params['total_time_steps']
    num_encoder_steps = params['num_encoder_steps']
    column_definition = params["column_definition"]


    data_csv_path = "{}.csv".format(exp_name)
    raw_data = pd.read_csv(data_csv_path)

    data = formatter.transform_data(raw_data)
    train_max, valid_max = formatter.get_num_samples_for_calibration(num_train=batch_size)
    max_samples = (train_max, valid_max)

    _, _, test = batch_sampled_data(data, 0.8, max_samples, params['total_time_steps'],
                                            params['num_encoder_steps'], pred_len,
                                            params["column_definition"],
                                            batch_size)

    test_enc, test_dec, test_y = next(iter(test))
    total_b = len(list(iter(test)))


    model_path = "models_{}_{}".format(exp_name, pred_len)
    model_params = formatter.get_default_model_params()

    src_input_size = test_enc.shape[2]
    tgt_input_size = test_dec.shape[2]

    predictions = np.zeros((3, total_b, test_y.shape[0], test_y.shape[1]))
    test_y_tot = torch.zeros((total_b, test_y.shape[0], test_y.shape[1]))
    n_batches_test = test_enc.shape[0]


    mse = nn.MSELoss()
    mae = nn.L1Loss()
    stack_size = 1

    seed_ls = []
    i = -1
    for f in os.listdir(model_path):
      parts = f.split("_")
      
      if model_name in f:
        i += 1

        seed = int(parts[-1])
        for d in d_model:
              try:
                  
                  d_k = int(d / n_heads)

                  config = src_input_size, tgt_input_size, d, n_heads, d_k, stack_size

                  model = Forecast_denoising(
                                      pred_len=pred_len,
                                      config=config,
                                      model_name=model_name,
                                      device=device,
                                      attn_type=attn_type,
                                      seed=seed,
                                      denoise=denoising, gp=gp)

                  checkpoint = torch.load(os.path.join("models_{}_{}".format(exp_name, pred_len),
                                          "{}_{}".format(model_name, seed)))
                  state_dict = checkpoint['model_state_dict']
                  new_state_dict = OrderedDict()

                  model.load_state_dict(checkpoint['model_state_dict'])

                  model.eval()
                  model.to(device)

                  j = 0
                  for test_enc, test_dec, test_y in test:
                      if denoising:
                          output, _ = model(test_enc.to(device), test_dec.to(device))
                      else:
                          output = model(test_enc.to(device), test_dec.to(device))

                      predictions[i, j] = output.squeeze(-1).cpu().detach().numpy()
                      if i == 0:
                          test_y_tot[j] = test_y.squeeze(-1).cpu().detach()
                      j += 1

              except RuntimeError as e:
                    pass
        

    predictions = torch.from_numpy(np.mean(predictions, axis=0))

    results = torch.zeros(2, pred_len)
    normaliser = test_y_tot.abs().mean()

    mse_loss = mse(predictions, test_y_tot).item() / normaliser
    mae_loss = mae(predictions, test_y_tot).item() / normaliser

    print("{}: MSE: {:.3f}".format(model_name, mse_loss))
    print("{}: MAE: {:.3f}".format(model_name, mae_loss))

In [102]:
evaluate("ATA_gp", True, True)
evaluate("ATA_iso", True, False)
evaluate("ATA_no", False, False)

Using root folder /content/Corruption-resilient-Forecasting-Models/outputs
Formatting data.
Setting scalers with data_set...
ATA_gp: MSE: 0.493
ATA_gp: MAE: 0.584
Using root folder /content/Corruption-resilient-Forecasting-Models/outputs
Formatting data.
Setting scalers with data_set...
ATA_iso: MSE: 0.513
ATA_iso: MAE: 0.591
Using root folder /content/Corruption-resilient-Forecasting-Models/outputs
Formatting data.
Setting scalers with data_set...
ATA_no: MSE: 0.570
ATA_no: MAE: 0.636
